In [41]:
import json
import pandas as pd
import gurobipy as gp
from datetime import datetime, timedelta

In [42]:
OBP_AVG = 0.327
SLG_AVG = 0.365

FPCT_AVG = {
    '2': 0.992,
    '3': 0.993,
    '4': 0.977, 
    '5': 0.929,
    '6': 0.964,
    '7': 0.977,
    '8': 0.988,
    '9': 0.976
}

POSITION_WEIGHT = {
    '2': {"batting": 0.95, "fielding": 1.05},
    '3': {"batting": 1.21, "fielding": 0.83},
    '4': {"batting": 0.97, "fielding": 1.03},
    '5': {"batting": 0.96, "fielding": 1.04},
    '6': {"batting": 0.94, "fielding": 1.06},
    '7': {"batting": 1.02, "fielding": 0.98},
    '8': {"batting": 1.04, "fielding": 0.96},
    '9': {"batting": 1.17, "fielding": 0.85},
}

OPPONENT_WEIGHT = {
    "中信兄弟": 1.15,
    "統一7-ELEVEn獅": 1.11,
    "樂天桃猿": 0.96, 
    "富邦悍將": 0.93,
    "味全龍": 0.85
}

dataRoot = "../clean/clean_all_"
playerRoot = "../球員異動/"

teamNameMap = {
    "中信兄弟": "brothers", 
    "味全龍": "dragons",
    "富邦悍將": "guardians", 
    "統一7-ELEVEn獅": "lions",
    "樂天桃猿": "monkeys"
}

### Import Data

In [43]:
class Player:
    def __init__(self, name, data):
        self.name = name
        self.data = data

In [44]:
allData = {}
for team in teamNameMap.keys():
    with open(dataRoot + team + ".json", encoding="utf-8") as f:
        temp = []
        for line in f:
            player = json.loads(line)
            temp.append(Player(player["name"], player["data"]))
        allData[teamNameMap[team]] = temp

brothers = allData["brothers"]
dragons = allData["dragons"]
guardians = allData["guardians"]
lions = allData["lions"]
monkeys = allData["monkeys"]

In [45]:
playerList = {}

for team in teamNameMap.keys():
    tempDict = pd.read_csv(playerRoot + team + ".csv", header=0, index_col=0).to_dict()
    tempDict = {
        datetime.strptime(key, "%Y-%m-%d").date(): list(filter(lambda d: pd.isna(d) == False, list(value.values())))
        for key, value in tempDict.items()
    }
    playerList[teamNameMap[team]] = tempDict

In [46]:
def createDate(mm, dd):
    return datetime(2022, mm, dd, 0, 0, 0).date()

### Self-defined Variables

In [47]:
aTeam = "味全龍"
aMonth = "May"
aGame = [
    {
        "date": createDate(4, 12),
        "field": "洲際", 
        "oppo": "中信兄弟", 
        "pitcher": "鄭凱文"
    }, 
    {
        "date":createDate(4, 14), 
        "field": "澄清湖", 
        "oppo": "統一7-ELEVEn獅", 
        "pitcher": "胡智為"
    }, 
    {
        "date": createDate(4, 15), 
        "field": "天母", 
        "oppo": "樂天桃猿", 
        "pitcher": "陳冠宇"
    }, 
    {
        "date": createDate(4, 16), 
        "field": "天母", 
        "oppo": "樂天桃猿", 
        "pitcher": "霸林爵"
    }, 
    {
        "date": createDate(4, 17), 
        "field": "天母", 
        "oppo": "樂天桃猿", 
        "pitcher": "王溢正"
    }
]
# aStar = ["許基宏", "江坤宇"]
# aPotential = ["岳東華"]
# aInjured = ["周思齊"]

### Calculate Parameters

In [48]:
# 比賽當天 27 人名單去除投手
def findPlayerList(team, date):
    minDelta = timedelta(days=100)
    playerDay = date
    for key in playerList[teamNameMap[aTeam]].keys():
        if date > key:
            break
        if key - date < minDelta:
            minDelta = key - date
            playerDay = key
    
    withoutPitcherList = []
    for data in allData[teamNameMap[team]]:
        withoutPitcherList.append(data.name)
    return list(filter(lambda d: d in withoutPitcherList, playerList[teamNameMap[team]][playerDay]))
        

In [49]:
# A_{ij}
def calcA(player, pos):
    if str(pos) in player.data["fielding"]["pos"].keys():
        return 1
    else:
        return 0

In [50]:
# F_{ij}
def calcF(player, pos):
    if str(pos) in player.data["fielding"]["pos"].keys():
        F = float(player.data["fielding"]["pos"][str(pos)]["FPCT"]) / FPCT_AVG[str(pos)]
        if F == 1:
            return 0.98
        if F < 0.8:
            return 0.8
        # print(player.name, "F:", F)
        return F
    else:
        return 0

In [51]:
# B_i
def calcOPS(OBP, SLG):
    return OBP / OBP_AVG + SLG / SLG_AVG - 1

def calcB(game, player):
    if game["date"].month == 4:
        aMonth = "Apr"
    if game["date"].month == 5:
        aMonth = "May"
    if game["date"].month == 6:
        aMonth = "Jun"
    aOppo = game["oppo"]
    aPitcher = game["pitcher"]
    aField = game["field"]

    OPSseason = float(player.data["batting"]["season"]["OPS+"])
    PAseason = int(player.data["batting"]["season"]["PA"])
    
    if "month" in player.data["batting"].keys():
        if aMonth in player.data["batting"]["month"].keys():
            OPSmonth = calcOPS(
                float(player.data["batting"]["month"][aMonth]["OBP"]), 
                float(player.data["batting"]["month"][aMonth]["TB"]) / float(player.data["batting"]["month"][aMonth]["AB"])
            )
            PAmonth = int(player.data["batting"]["month"][aMonth]["PA"])
        else:
            OPSmonth = OPSseason
            PAmonth = 0
    else:
        OPSmonth = OPSseason
        PAmonth = 0

    if "field" in player.data["batting"]:
        if aField in player.data["batting"]["field"].keys():
            OPSfield = calcOPS(
                float(player.data["batting"]["field"][aField]["OBP"]), 
                float(player.data["batting"]["field"][aField]["TB"]) / float(player.data["batting"]["field"][aField]["AB"])
            )
            PAfield = int(player.data["batting"]["field"][aField]["PA"])
        else:
            OPSfield = OPSseason
            PAfield = 0
    else:
        OPSfield = OPSseason
        PAfield = 0
    
    if "vsP" in player.data["batting"].keys():
        if aOppo in player.data["batting"]["vsP"]["data"].keys():
            if aPitcher in player.data["batting"]["vsP"]["data"][aOppo].keys():
                if player.data["batting"]["vsP"]["data"][aOppo][aPitcher]["AB"] != "0":
                    OPSvsp = calcOPS(
                        float(player.data["batting"]["vsP"]["data"][aOppo][aPitcher]["OBP"]), 
                        float(player.data["batting"]["vsP"]["data"][aOppo][aPitcher]["TB"]) / float(player.data["batting"]["vsP"]["data"][aOppo][aPitcher]["AB"])
                    )
                    PAvsp = int(player.data["batting"]["vsP"]["data"][aOppo][aPitcher]["PA"])
                else:
                    OPSvsp = 0
                    PAvsp = 0
            else:
                OPSvsp = 0
                PAvsp = 0
        else:
            OPSvsp = OPSseason
            PAvsp = 0
    else:
        OPSvsp = OPSseason
        PAvsp = 0

    B = (OPSseason * PAseason + OPSmonth * PAmonth + OPSfield * PAfield + OPSvsp * PAvsp) / 400
    # print(player.name, B)
    return B

# V_{ij}
def calcV(game, player, pos):
    A = calcA(player, pos)
    B = calcB(game, player)
    F = calcF(player, pos)
    # print(player.name, "V:", (POSITION_WEIGHT[str(pos)]["batting"] * B + POSITION_WEIGHT[str(pos)]["fielding"] * F) * A)
    return (POSITION_WEIGHT[str(pos)]["batting"] * B + POSITION_WEIGHT[str(pos)]["fielding"] * F) * A

### Optimization

In [52]:
model = gp.Model("model")

# decision variables
x = []
for g in range(len(aGame)):
    todayPlayerList = findPlayerList(aTeam, aGame[g]["date"])
    singleGame = {}
    for i in range(len(todayPlayerList)):
        singlePlayer = []
        for j in range(2, 10):
            singlePlayer.append(model.addVar(lb=0, vtype="B", name="x" + str(g) + "-" + todayPlayerList[i] + "-" + str(j)))
        singleGame[todayPlayerList[i]] = singlePlayer
    x.append(singleGame)

# objective function
model.setObjective(gp.quicksum(
    gp.quicksum(
        gp.quicksum(
            OPPONENT_WEIGHT[aGame[g]["oppo"]] * calcV(aGame[g], list(filter(lambda d: d.name == thisPlayer, allData[teamNameMap[aTeam]]))[0], j) * x[g][thisPlayer][j - 2]
            for j in range(2, 10)
        ) for thisPlayer in findPlayerList(aTeam, aGame[g]["date"])
    ) for g in range(len(aGame))
), gp.GRB.MAXIMIZE)

# constraints
for g in range(len(aGame)):
    todayPlayerList = findPlayerList(aTeam, aGame[g]["date"])
    for thisPlayerName in todayPlayerList:
        thisPlayer = list(filter(lambda d: d.name == thisPlayerName, allData[teamNameMap[aTeam]]))[0]
        
        # constraint 1
        model.addConstr(calcF(thisPlayer, 8) * x[g][thisPlayerName][8 - 2] >= calcF(thisPlayer, 7) * x[g][thisPlayerName][7 - 2])
        # constraint 2
        model.addConstr(calcF(thisPlayer, 8) * x[g][thisPlayerName][8 - 2] >= calcF(thisPlayer, 9) * x[g][thisPlayerName][9 - 2])
        
        # constraint 4
        model.addConstr(gp.quicksum(x[g][thisPlayerName][j - 2] for j in range(2, 10)) == 1)

    for j in range(2, 10):
        # constraint 3
        model.addConstr(gp.quicksum(x[g][thisPlayerName][j - 2] for thisPlayerName in todayPlayerList) == 1)

model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 265 rows, 600 columns and 1275 nonzeros
Model fingerprint: 0xd1d7245d
Variable types: 0 continuous, 600 integer (600 binary)
Coefficient statistics:
  Matrix range     [9e-01, 1e+00]
  Objective range  [3e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 150 rows and 45 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible
Best objective -, best bound -, gap -


In [53]:
for var in model.getVars():
    if var.x != 0.0:
        print(var.varName, var.x)

AttributeError: Unable to retrieve attribute 'x'